Корпоратив
Все мы часто задаемся вопросом, где нам покушать? Предположим Вы подчиненный владельца крупной компании с (кхм)жёлтым приложением. Ваш босс хочет провести корпоратив для всего технического отдела. Он предложил Вам провести исследование, где лучше всего расположиться Вашей большой командой. Рассчитывайте, что Вас не менее 350 человек. Бюджет не ограничен. Подберите место, которое по расположению ближе всего к Красной площади и при этом вмещает нужное кол-во человек. При этом он не хочет праздновать в какой-то там столовой.
Для решения задачи постройте карту, нанесите на неё заведения, удовлетворяющие пожеланиям босса. В конце проведите некоторый анализ и приведите аргументы, почему Вы выбрали то или иное заведение.
В анализ входит:
 1. Показать вероятность того, что если босс случайно выберет заведение из списка, удовлетворяющего условию, оно окажется рестораном. Сравните эту вероятность с другими видами объектов
 2. Определите топ 30 ресторанов Москвы по расположению, считаем, если мы близко к центру, то это заведение круче, чем остальные

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='whitegrid', font_scale=1.3, palette='Set2')